In [74]:
import numpy as np
import pandas as pd
import torch
import json

In [75]:
# load model
H = torch.load("H_rank100.pt",map_location=torch.device('cpu'))
W = torch.load("W_rank100.pt",map_location=torch.device('cpu'))

In [106]:
# load book/music indices
books_ix = pd.read_csv("books_ix.csv", header=None)
books_ix = books_ix.drop_duplicates()
music_meta = pd.read_csv("combinedMusicData_small.csv", header=None)[[0, 1, 5]]

In [107]:
music_meta2 = music_meta.copy()
music_meta2.columns = ["title", 'tag', 'uri']
music_meta2.head()

,title,tag,uri
0,Can I Live,rap,spotify:track:20U1I32q6kX0Nmy4u8CvUg
1,Forgive Me Father,rap,spotify:track:1cvzmIBhdaQdC1DBbFnpWB
2,Down and Out,rap,spotify:track:5mEwhCsqadBZIE7SSrv1gC
3,Fly In,rap,spotify:track:6Nr3QjImPUhCpUgnykO7hm
4,Im Not You,rap,spotify:track:4h1xaUGnbHp867AZ4mBM3m


In [86]:
books_ix.to_csv("book_data.csv")

In [78]:
def createRow(query_ix):
    song_scores = (W[query_ix] @ H.T)
    top_song_ix = np.argsort(song_scores.detach().numpy())[::-1][:1000]
    
    srt = np.sort(top_song_ix)+1
    
    music_df = music_meta.loc[srt]
    
    music_df.columns = ["title", 'tag', 'uri']
    music_df['score'] = song_scores.detach().numpy()[np.sort(top_song_ix)]
    
    music_df = music_df.sort_values('score',ascending=False)
    music_df = music_df[music_df.tag != 'misc']
    
    output = {}
    
    default_df = music_df[:15]
    
    output['name'] = books_ix.iloc[query_ix].item()
    output['default'] = default_df['uri'].tolist()

    for genre in music_df['tag'].value_counts().index.values:
        playlist_df = music_df[music_df['tag'] == genre][:15]
        if len(playlist_df) >= 15 and genre:
            output[genre] = playlist_df['uri'].tolist()
            
    
    return output

In [80]:
def makeJson(startIdx, endIdx):
    output = []
    for i in range(startIdx, endIdx):
        d = createRow(i)
        output.append(d)

    json_object = json.dumps(output, indent=4)

    with open(f'data_{startIdx}.json', 'w', newline='') as outfile:
        outfile.write(json_object)

In [81]:
makeJson(0, 3300)

In [82]:
makeJson(3300, 6600)

In [83]:
makeJson(6600, 9900)

In [84]:
makeJson(9900, 13200)

In [85]:
makeJson(13200, 16277)